In [1]:
import os
import math

import numpy as np
import tensorflow as tf

slim = tf.contrib.slim

In [2]:
import sys
sys.path.append('../')

In [3]:
from models_slim import inception, inception_utils
from models_slim import inception_v1, inception_v2, inception_v3, inception_v4

In [4]:
# TensorFlow session: grow memory when needed. TF, DO NOT USE ALL MY GPU MEMORY!!!
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options)
isess = tf.InteractiveSession(config=config)

# Inception checkpoints

Add scale parameter to be able to use fused-BN.

In [5]:
def restore_checkpoint(sess, ckpt_filename, moving_average_decay=0.9999):
    tf_global_step = slim.get_or_create_global_step()
    # Restore moving average variables or classic stuff!
    if moving_average_decay:
        variable_averages = tf.train.ExponentialMovingAverage(
            moving_average_decay, tf_global_step)
        variables_to_restore = variable_averages.variables_to_restore(
            tf.contrib.framework.get_model_variables())
        variables_to_restore[tf_global_step.op.name] = tf_global_step
    else:
        variables_to_restore = tf.contrib.framework.get_variables_to_restore()
    # Restore method.
    fn_restore = slim.assign_from_checkpoint_fn(ckpt_filename,
                                                variables_to_restore,
                                                ignore_missing_vars=True)
    fn_restore(sess)

In [12]:
inception_model = inception.Inceptionv4Model()
ckpt_filename = '../checkpoints/inception_v4.ckpt'
        
with tf.Graph().as_default():
    # Input placeholder.
    data_format = 'NCHW'
    imgsize = inception_model.image_size
    net_shape = (imgsize, imgsize)
    img_input = tf.placeholder(tf.float32, shape=(1, 3, imgsize, imgsize))
    
    # Network...
    output = inception_model.inference(img_input, num_classes=1001, is_training=False, 
                                       data_format=data_format, data_type=tf.float32)
    
    with tf.Session() as sess:
        # Initialize variables.
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        # Load checkpoint. Should WARNINGS because of missing scale parameter...
        restore_checkpoint(sess, ckpt_filename, moving_average_decay=None)
        # Save back checkpoint!
        save_filename = ckpt_filename.replace('.ckpt', '_fused.ckpt')
        saver = tf.train.Saver()
        saver.save(sess, save_filename, write_meta_graph=False)

INFO:tensorflow:Restoring parameters from ../checkpoints/inception_v4.ckpt
